In [68]:
import json
import re
from konlpy.tag import Okt
from collections import Counter
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
from wordcloud import WordCloud
import pandas as pd

import glob
from functools import reduce
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from wordcloud import WordCloud, STOPWORDS

import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim 
import warnings
warnings.filterwarnings('ignore')

AttributeError: module 'tweepy' has no attribute 'StreamListener'

In [ ]:
import pandas as pd

# 데이터 정리
### 영화정보
> meta19 meta20 meta21

### 리뷰
> 다음 : d19 | d20 | d21<br>

# 데이터 병합

In [89]:
d19 = pd.read_csv('./data/review/daum_review2019_last.csv')

In [90]:
d20 = pd.read_csv('./data/review/daum_review2020_last.csv')

In [91]:
d21 = pd.read_csv('./data/review/daum_review2021_last.csv')

In [92]:
d21['release_date']='null'
d21 = d21[['title','score','content','write_date','release_date','source_site']]

In [93]:
print(d19.columns)
print(d20.columns)
print(d21.columns)

d21.head()

Index(['title', 'score', 'content', 'write_date', 'release_date',
       'source_site'],
      dtype='object')
Index(['title', 'score', 'content', 'write_date', 'release_date',
       'source_site'],
      dtype='object')
Index(['title', 'score', 'content', 'write_date', 'release_date',
       'source_site'],
      dtype='object')


,title,score,content,write_date,release_date,source_site
0,루카,10,싱그러운 바닷가 코로나시국에 맑은 자연이 그립다.,2021. 01. 07. 02:40,null,daum
1,루카,10,랜선 이탈리아 바다 여행 느낌....힐링힐링 빨리 개봉했으면,2021. 06. 15. 11:03,null,daum
2,루카,9,뻔한 스토리 같지만 뻔하지 않은 감동이 있었슴.마지막엔 눈가를 적실만한 감동도......,2021. 06. 21. 10:12,null,daum
3,루카,9,아들들이랑 정말 잼있게 봤네요 재미도 있고 감동도 있고 아름다운 영상미에 시원한 ...,2021. 06. 20. 23:38,null,daum
4,루카,10,픽사는사람을 울게 만든다니깐~,2021. 06. 21. 22:46,null,daum


In [94]:
d21.head(1)

,title,score,content,write_date,release_date,source_site
0,루카,10,싱그러운 바닷가 코로나시국에 맑은 자연이 그립다.,2021. 01. 07. 02:40,null,daum


In [114]:
d1920 = pd.merge(d19, d20, on=('title','score','content','write_date','release_date','source_site'),how='outer')

In [115]:
d192021 = pd.merge(d1920, d21, on=('title','score','content','write_date','release_date', 'source_site'),how='outer')

In [116]:
meta192021 = pd.read_csv('./data/review/meta192021.csv')

In [117]:
d192021 = d192021.reset_index(drop=True)

In [118]:
d192021.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51773 entries, 0 to 51772
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         51773 non-null  object
 1   score         51773 non-null  int64 
 2   content       49126 non-null  object
 3   write_date    51773 non-null  object
 4   release_date  23249 non-null  object
 5   source_site   51773 non-null  object
dtypes: int64(1), object(5)
memory usage: 2.4+ MB


# 리뷰 결측치 제거

In [119]:
d192021 = d192021[d192021['content'].isnull() == 0]

# 개봉일 결측치 채우기 

In [120]:
d192021 = d192021.fillna('null')

In [121]:
print(len(d192021))
d192021 = d192021.drop_duplicates(subset='content').reset_index(drop=True)
print(len(d192021))

49126
27399


### csv 생성

In [84]:
# d192021.to_csv('./data/total/Daum_review_192021.csv',index=False)

In [132]:
d192021 = d192021.drop('release_date',1)

/tmp/ipykernel_16192/615102753.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  d192021 = d192021.drop('release_date',1)


In [138]:
movie_all = pd.merge(meta192021, d192021
    , how = 'inner'
    , on = "title")

In [139]:
movie_all = movie_all.drop_duplicates(subset='content').reset_index(drop=True)
movie_all.to_csv('./data/total/Daum_review_192021.csv',index=False)

In [79]:
movie_all = d192021.drop_duplicates(subset='content').reset_index(drop=True)

---

In [58]:
movie_all = movie_all.drop('release_date_y',axis=1)

In [59]:
movie_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48919 entries, 0 to 48918
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   title           48919 non-null  object
 1   genre           48919 non-null  object
 2   nation          48919 non-null  object
 3   running_time    48919 non-null  object
 4   release_date_x  48919 non-null  object
 5   director        48919 non-null  object
 6   actor           48919 non-null  object
 7   grade           48919 non-null  object
 8   summary         48919 non-null  object
 9   poster          48919 non-null  object
 10  score           48919 non-null  int64 
 11  content         48919 non-null  object
 12  write_date      48919 non-null  object
 13  source_site     48919 non-null  object
dtypes: int64(1), object(13)
memory usage: 6.6+ MB


In [76]:
movie_all.title.nunique()

92

In [64]:
movie_all

,title,genre,nation,running_time,release_date_x,director,actor,grade,summary,poster,score,content,write_date,source_site
0,드래곤 길들이기 3,"애니메이션, 모험",미국,104분,2019 .01.30 개봉,딘 데블로이스,"제이 바루첼(히컵 목소리), 아메리카 페레라(아스트리드 목소리), 케이트 블란쳇(발...",전체 관람가,영원한 친구 히컵과 투슬리스의 활약으로\n 사람과 드래곤이 공존하며 평화롭게 살아가...,https://movie-phinf.pstatic.net/20190131_168/1...,10,재밌음,2020. 11. 26. 15:35,daum
1,드래곤 길들이기 3,"애니메이션, 모험",미국,104분,2019 .01.30 개봉,딘 데블로이스,"제이 바루첼(히컵 목소리), 아메리카 페레라(아스트리드 목소리), 케이트 블란쳇(발...",전체 관람가,영원한 친구 히컵과 투슬리스의 활약으로\n 사람과 드래곤이 공존하며 평화롭게 살아가...,https://movie-phinf.pstatic.net/20190131_168/1...,8,"인간과 드래곤의 공생, 진정한 유토피아를 찾아서. 마지막 장면은 토이스토리4를 연상...",2020. 09. 27. 23:39,daum
2,드래곤 길들이기 3,"애니메이션, 모험",미국,104분,2019 .01.30 개봉,딘 데블로이스,"제이 바루첼(히컵 목소리), 아메리카 페레라(아스트리드 목소리), 케이트 블란쳇(발...",전체 관람가,영원한 친구 히컵과 투슬리스의 활약으로\n 사람과 드래곤이 공존하며 평화롭게 살아가...,https://movie-phinf.pstatic.net/20190131_168/1...,10,엄청 재밌고 라이트 퓨리 너무 귀여워요,2020. 09. 10. 16:39,daum
3,드래곤 길들이기 3,"애니메이션, 모험",미국,104분,2019 .01.30 개봉,딘 데블로이스,"제이 바루첼(히컵 목소리), 아메리카 페레라(아스트리드 목소리), 케이트 블란쳇(발...",전체 관람가,영원한 친구 히컵과 투슬리스의 활약으로\n 사람과 드래곤이 공존하며 평화롭게 살아가...,https://movie-phinf.pstatic.net/20190131_168/1...,9,보고또봐도 재밌고 뭉클,2020. 11. 28. 14:21,daum
4,드래곤 길들이기 3,"애니메이션, 모험",미국,104분,2019 .01.30 개봉,딘 데블로이스,"제이 바루첼(히컵 목소리), 아메리카 페레라(아스트리드 목소리), 케이트 블란쳇(발...",전체 관람가,영원한 친구 히컵과 투슬리스의 활약으로\n 사람과 드래곤이 공존하며 평화롭게 살아가...,https://movie-phinf.pstatic.net/20190131_168/1...,10,재밌어요!,2020. 05. 07. 10:52,daum
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48914,프리 가이,"액션, 모험",미국,115분,2021 .08.11 개봉,숀 레비,라이언 레이놀즈(가이),12세 관람가,"평범한 직장, 절친 그리고 한 잔의 커피.\n 평화로운 일상 속 때론 총격전과 날...",https://movie-phinf.pstatic.net/20210721_243/1...,9,2주전에 벌써 p2p사이트에 많이 떠돌고 있던데...,2020. 02. 26. 02:04,daum
48915,프리 가이,"액션, 모험",미국,115분,2021 .08.11 개봉,숀 레비,라이언 레이놀즈(가이),12세 관람가,"평범한 직장, 절친 그리고 한 잔의 커피.\n 평화로운 일상 속 때론 총격전과 날...",https://movie-phinf.pstatic.net/20210721_243/1...,10,평점이 왜 다들 높은지....트루먼쇼를 게임화? 그래픽은 나름 괘안치만 다른건 그닥,2021. 08. 24. 13:20,daum
48916,프리 가이,"액션, 모험",미국,115분,2021 .08.11 개봉,숀 레비,라이언 레이놀즈(가이),12세 관람가,"평범한 직장, 절친 그리고 한 잔의 커피.\n 평화로운 일상 속 때론 총격전과 날...",https://movie-phinf.pstatic.net/20210721_243/1...,5,잤다. 현실과 게임사이를 왔다갔다하는 데에 이입이 힘들었다.,2021. 08. 16. 18:31,daum
48917,프리 가이,"액션, 모험",미국,115분,2021 .08.11 개봉,숀 레비,라이언 레이놀즈(가이),12세 관람가,"평범한 직장, 절친 그리고 한 잔의 커피.\n 평화로운 일상 속 때론 총격전과 날...",https://movie-phinf.pstatic.net/20210721_243/1...,1,뭔가 기발한 것 같은데 재미는 없고 역시나 난 미쿡식 유머는 진짜 안 맞는 것 같음...,2021. 08. 14. 18:41,daum


# 리뷰 중복값 제거 
* 22,171


In [73]:
movie_all = movie_all.drop_duplicates(subset='content').reset_index(drop=True)

In [77]:
movie_all.head(1)

,title,genre,nation,running_time,release_date_x,director,actor,grade,summary,poster,score,content,write_date,source_site
0,드래곤 길들이기 3,"애니메이션, 모험",미국,104분,2019 .01.30 개봉,딘 데블로이스,"제이 바루첼(히컵 목소리), 아메리카 페레라(아스트리드 목소리), 케이트 블란쳇(발...",전체 관람가,영원한 친구 히컵과 투슬리스의 활약으로\n 사람과 드래곤이 공존하며 평화롭게 살아가...,https://movie-phinf.pstatic.net/20190131_168/1...,10,재밌음,2020. 11. 26. 15:35,daum


In [78]:
movie_all = movie_all.reset_index(drop=True)

In [80]:
movie_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26748 entries, 0 to 26747
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   title           26748 non-null  object
 1   genre           26748 non-null  object
 2   nation          26748 non-null  object
 3   running_time    26748 non-null  object
 4   release_date_x  26748 non-null  object
 5   director        26748 non-null  object
 6   actor           26748 non-null  object
 7   grade           26748 non-null  object
 8   summary         26748 non-null  object
 9   poster          26748 non-null  object
 10  score           26748 non-null  int64 
 11  content         26748 non-null  object
 12  write_date      26748 non-null  object
 13  source_site     26748 non-null  object
dtypes: int64(1), object(13)
memory usage: 2.9+ MB


In [81]:
movie_all = movie_all.rename(columns={'release_date_x ':'release_date'})